# Исследование надежности заемщиков.


**Цель исследования** — проверить основную гипотезу:
Семейное положение и количество детей клиента влияет (положительно/отрицательно) на факт погашения кредита в срок.
Для подтвреждения или опровержения данной гипотезы будут даны ответы на ряд вопросов
1. Есть ли зависимость между количеством детей и возвратом кредита в срок.
2. Есть ли зависимость между семейным положением и возвратом кредита в срок.
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок.
4. Как разные цели кредита влияют на его возврат в срок.


**Ход исследования**

Данные для исследования взяты из файла `/datasets/data.csv`. О качестве данных ничего не известно. Поэтому перед проверкой гипотез понадобится обзор данных, предобработка, после чего можно будет перейти к проверке гипотезы.  


Исследование пройдёт в четыре этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотезы (Ответьте на вопросы).
 4. Выводы исследования.

### Шаг 1. Обзор данных

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('/datasets/data.csv')
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Сразу бросается в глаза, ряд странностей даже по тем 10 строчкам, что выведены:
1. В столбце days_employed присутствуют отрицательные значения - то есть как будто у кого-то общий трудовой стаж пошёл в обратную стороны
2. В столбце education значения написаны в разных стилях (все прописные/все строчные/camelcase) - при группировке возможны некорректные данные
3. Пары столбцов  education-education_id и family_status-family_status_id по факту показывают одну и ту же информацию, можно сделать словарь

Посмотрим общую информацию по таблице:

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Итак, в таблице 12 столбцов. 

Согласно документации к данным:
* `children` — количество детей в семье;
* `days_employed` — общий трудовой стаж в днях;  
* `dob_years` — возраст клиента в годах;
* `education` —уровень образования клиента;
* `education_id` — идентификатор уровня образования;
* `family_status` — семейное положен;  
* `family_status_id` — идентификатор семейного положения;
* `gender` — пол клиента;
* `income_type` — тип занятости;
* `debt ` — имел ли задолженность по возврату кредитов;
* `total_income` — ежемесячный доход;
* `purpose` — цель получения кредита.




Количество значений в столбцах различается. В частности в столбцах days_employed и total_income, заполним эти пропуски медианным значением


### Шаг 2.1 Заполнение пропусков

определим, сколько раз пропущен общий трудовой стаж в днях в нашей таблице

In [4]:
data['days_employed'].value_counts()

-327.685916     1
-1580.622577    1
-4122.460569    1
-2828.237691    1
-2636.090517    1
               ..
-7120.517564    1
-2146.884040    1
-881.454684     1
-794.666350     1
-3382.113891    1
Name: days_employed, Length: 19351, dtype: int64

Пока непонятно, попробуем по-другому

In [5]:
data[data['days_employed'].isna()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


In [6]:
# посчитаем колчиество пустых строк по столбцу
null_case =  data['days_employed'].isna().sum()
all_case = len(data.index)
print('Всего пустых значений по столбцу:', data['days_employed'].isna().sum())
print('Отношение пустых значений к общему:', null_case/all_case * 100 )

Всего пустых значений по столбцу: 2174
Отношение пустых значений к общему: 10.099883855981417


In [7]:
print('Отношение пустых значений к общему:', data['days_employed'].isna().mean()*100)

Отношение пустых значений к общему: 10.099883855981417


Пустые значения по столбцу "days_employed" есть (примерно 10 пероцентов в выборке), поменям на медианные, предварительно посчитав её

In [8]:
median_value = data['days_employed'].median()
median_value

-1203.369528770489

Медианное значение получилось отрицательным - это достаточно странное значение, поэтому для начала размебермся с этим и только потом заполним пропуски медианным значением.

In [9]:
print('Всего отрицательных значений:', data[data['days_employed'] < 0]['days_employed'].count())

Всего отрицательных значений: 15906


Проанализируем отрицательные значения в рабочих днях по каждому столбцу. Возможно для определённого значения в столбце всегда отсчёт считается отрицательным. Если в результате группировки мы увидим только одно значение это может быть предпоссылкой к тому, чтобы сделать вывод что при заполнении одного конкретного значения сделана ошибка

In [10]:
negative_data = data[data['days_employed'] < 0]
print(negative_data.groupby('education')['days_employed'].count())
print()
print(negative_data.groupby('family_status')['days_employed'].count())
print()
print(negative_data.groupby('gender')['days_employed'].count())
print()
print(negative_data.groupby('income_type')['days_employed'].count())
print()
print(negative_data.groupby('purpose')['days_employed'].count())
print()

education
ВЫСШЕЕ                  222
Высшее                  222
НАЧАЛЬНОЕ                 9
НЕОКОНЧЕННОЕ ВЫСШЕЕ      21
Начальное                10
Неоконченное высшее      37
СРЕДНЕЕ                 570
Среднее                 498
УЧЕНАЯ СТЕПЕНЬ            1
высшее                 3751
начальное               149
неоконченное высшее     582
среднее                9831
ученая степень            3
Name: days_employed, dtype: int64

family_status
Не женат / не замужем    2212
в разводе                 885
вдовец / вдова            381
гражданский брак         3158
женат / замужем          9270
Name: days_employed, dtype: int64

gender
F      9945
M      5960
XNA       1
Name: days_employed, dtype: int64

income_type
в декрете              1
госслужащий         1312
компаньон           4577
предприниматель        1
сотрудник          10014
студент                1
Name: days_employed, dtype: int64

purpose
автомобили                                340
автомобиль                         

Попытка не пытка. Зайдем с другом стороны посмотрим на людей с положительным стажем.

In [11]:
positive_data = data[data['days_employed'] > 0]
#посмотрим на того, кто работал больше всех
positive_data[positive_data['days_employed'] == positive_data['days_employed'].max()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
6954,0,401755.400475,56,среднее,1,вдовец / вдова,2,F,пенсионер,0,176278.441171,ремонт жилью


Человек проработал 401755 дней. То есть 1100 лет (401755/365), похоже у нас ошибка в данных, возможно неправильная единица измерения

Основной вывод. Данные  в источнике по столбцу "days_employed" предоставлены некорректно, если мы заполним отсутствующие данные медианными мы получим еще больше некорректных данных. По итогу надо сделать анализ без использования этого столбца.

Посмотрим какая ситуация со столбцом "total_income"

In [12]:
print('Всего пустых значений по столбцу:', data['total_income'].isna().sum())

Всего пустых значений по столбцу: 2174


In [13]:
median_value_total_income = data['total_income'].median()
median_value_total_income

145017.93753253992

Значение намного более адекватное (не отрицательное). На всякий случай проверим есть ли в выборке строки с отрицательным доходом

In [14]:
 data[data['total_income'] < 0]['total_income'].count()

0

Отлично, значит можно заменить значения по пустым на медианные

Пустые значения в количественных переменных правильней заполнять медианным значеним, т.к. в таком случае мы избегаем вбросы значений(слишком большие или слишком маленькие) если размах значений в выборке (самое большое значение минус самое минимальное) излишне велик.
Например, нельзя взять среднеарифетическое значение по зарплате при составлении статистики по России, у чиновника зарплата миллион(и это только официально) у врача 15000 в среднем они получают 507 500, что с математической точки зрения верно, но не соответствует реальному положению дел.

In [15]:
data['total_income'] = data['total_income'].fillna(value=median_value_total_income)

Доверяй, но проверяй - проверим остались ли пустые

In [16]:
print('Всего пустых значений по столбцу:', data['total_income'].isna().sum())

Всего пустых значений по столбцу: 0


### Шаг 2.2 Проверка данных на аномалии и исправления.

In [17]:
#посмотрим уникальные значения по столбцу "children"
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

Из того что мы видим выбиваются два значения -1(?) и 20(слишком много). Посмотрим детальней

In [18]:
data[data['children']==-1]['children'].count()

47

Возможно эта ситуация предполагает, что ребенок был но умер. Для нашего анализа отрицательное количество детей и их отсутствие идентичны с точки зрения анализа, поэтому присвоим всем -1 значение 0

In [19]:
data.loc[data['children'] == -1, 'children'] = 0

In [20]:
#посмотрим скольео записей с количеством детей равном 42

data[data['children'] == 20]['children'].count()
data[data['children'] == 20]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,-880.221113,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья
720,20,-855.595512,44,среднее,1,женат / замужем,0,F,компаньон,0,112998.738649,покупка недвижимости
1074,20,-3310.411598,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518.537004,получение образования
2510,20,-2714.161249,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474.835577,операции с коммерческой недвижимостью
2941,20,-2161.591519,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739.941398,на покупку автомобиля
...,...,...,...,...,...,...,...,...,...,...,...,...
21008,20,-1240.257910,40,среднее,1,женат / замужем,0,F,сотрудник,1,133524.010303,свой автомобиль
21325,20,-601.174883,37,среднее,1,женат / замужем,0,F,компаньон,0,102986.065978,профильное образование
21390,20,NaN,53,среднее,1,женат / замужем,0,M,компаньон,0,145017.937533,покупка жилой недвижимости
21404,20,-494.788448,52,среднее,1,женат / замужем,0,M,компаньон,0,156629.683642,операции со своей недвижимостью


Учитывая что есть строчка с количеством детей 20 и возрастом 21, а также что нету никаких проежуточных значений между 20 и 5, мы предполагаем, что в данных закралась ошибка, и на самом деле детей 2, а не 20, поэтому поменяем значение 20 на 2

In [21]:
data.loc[data['children'] == 20, 'children'] = 2

In [22]:
#посмотрим теперь уникальные значения по столбцу "children"
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

Рассмотрим столбец dob_years

In [ ]:
#посмотрим уникальные значения по столбцу "dob_years", для удобства отсортируем
sorted = data.sort_values(by='dob_years', ascending = True)
sorted['dob_years'].unique()

Аномальным значением является значение 0

In [ ]:
anomal_dob_years = data[data['dob_years'] == 0]
#посмотрим в разбивке по образованию
anomal_dob_years.groupby('education')['dob_years'].count()

In [ ]:
#преобразуем в нижний регистр убрав тем самым цдалив неявные дубликаты
anomal_dob_years['education']=anomal_dob_years['education'].str.lower()

#сделаем такде на основном датафрейме 
data['education']=data['education'].str.lower()

In [ ]:
anomal_dob_years.groupby(['education'])['dob_years'].count()

Возможно при заполнении данных не было строгих правил, ограничивающих полтьзователя в заполнении ячеек или же человек готовивший выборку собирал её из разных источников, а разные источники - разные принципы хранения данных.

Какого-то конкретного объяснения это не выявило. Сделаем по аналогии с total_income

In [ ]:
median_value_dob_years = data['dob_years'].median()
median_value_dob_years
data.loc[data['dob_years'] == 0, 'children'] = median_value_dob_years

Cтолбец education мы отредактировали и убрали от дубликатов убрав всё в нижний регистр

Рассмотрим столбец family_status

In [ ]:
data['family_status'].unique()

In [29]:
# приведём всё в нижний регистр для однообразия данных
data['family_status']=data['family_status'].str.lower()

Рассмотрим столбец gender

In [30]:
data['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

In [31]:
anomal_gender = data[data['gender']=='XNA']
anomal_gender

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0.0,-2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


посчитаем сколько мужчин/женщин в выборке подходящим категориальным критериям

In [32]:
data[(data['family_status']=='гражданский брак') & (data['income_type']=='компаньон')& (data['education_id']==2)] .groupby(['gender'])['gender'].count()

gender
F      40
M      21
XNA     1
Name: gender, dtype: int64

Так как наиболее встречающийся пол по заданным парметрам женский, назначим данной строке женский пол
purpose намеренно пропустили

In [33]:
data.loc[data['gender'] == 'XNA', 'gender'] = 'F'

In [34]:
data['gender'].unique()

array(['F', 'M'], dtype=object)

Рассмотрим столбец income_type

In [35]:
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

Рассмотрим столбец debt

In [36]:
data['debt'].unique()

array([0, 1])

Рассмотрим столбец purpose

In [37]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

    Аномальных значений нет, но одинаковые по смыслу значения присутствуют, уберем их когда будем убирать дубликаты

### Шаг 2.3. Изменение типов данных.

Заменим тип данных в столбце total_income

In [38]:
data['total_income'] = data['total_income'].astype('int')

### Шаг 2.4. Удаление дубликатов.

In [39]:
data['children'].value_counts()

0.0     14127
1.0      4802
2.0      2117
3.0       328
42.0      101
4.0        41
5.0         9
Name: children, dtype: int64

Откуда-то появилось количество детей равное 42, был выброс -1 и 20, которые мы заменили, заменим и это. Откуда это появилось непонятно, но предобратобать это необходимо

In [40]:
data.loc[data['children'] == 42, 'children'] = 4

In [41]:
data['children'].value_counts()

0.0    14127
1.0     4802
2.0     2117
3.0      328
4.0      142
5.0        9
Name: children, dtype: int64

In [42]:
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Неявные дубликаты мы удалили когда преобразовали значения столбца по образованию в нижний регистр. 

In [43]:
#удалим явные дубликаты
data = data.drop_duplicates()

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создадим два новых датафрейма, в которых:
	
    каждому уникальному значению из education соответствует уникальное значение education_id — в первом;
	каждому уникальному значению из family_status соответствует уникальное значение family_status_id — во втором.

In [44]:
education = data.iloc[:,3:5]
education = education.drop_duplicates()
education

,education,education_id
0,высшее,0
1,среднее,1
13,неоконченное высшее,2
31,начальное,3
2963,ученая степень,4


In [45]:
family_status = data.iloc[:,5:7]
family_status = family_status.drop_duplicates()
family_status

,family_status,family_status_id
0,женат / замужем,0
4,гражданский брак,1
18,вдовец / вдова,2
19,в разводе,3
24,не женат / не замужем,4


Удалим столбцы education и family_status, оставив только их идентификаторы: education_id и family_status_id

In [46]:
data = data.drop(columns = ['education', 'family_status'], axis = 1) 

### Шаг 2.6. Категоризация дохода.

Напишем функцию по категоризации дохода

In [47]:
def catagorize_income(income):
    if income <= 30000:
        category = 'E'
        return category
    elif 50000 >= income > 30000:
        category = 'D'
        return category
    elif 200000 >= income > 50000:
        category = 'C'
        return category
    elif 1000000 >= income > 200000:
        category = 'B'
        return category
    else:
        category = 'A'
        return category

In [48]:
data['total_income_category'] = data['total_income'].apply(catagorize_income) 
data.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1.0,-8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1.0,-4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0.0,-5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3.0,-4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0.0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C
5,0.0,-926.185831,27,0,1,M,компаньон,0,255763,покупка жилья,B
6,0.0,-2879.202052,43,0,0,F,компаньон,0,240525,операции с жильем,B
7,0.0,-152.779569,50,1,0,M,сотрудник,0,135823,образование,C
8,2.0,-6929.865299,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C
9,0.0,-2188.756445,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C


### Шаг 2.7. Категоризация целей кредита.

Напишем функцию по категоризации целей кредита

In [49]:
def catagorize_purpose(purpose):
    if 'авто' in purpose:
        category = 'операции с автомобилем'
        return category
    elif 'свадьб' in purpose:
        category = 'проведение свадьбы'
        return category
    elif 'образован' in purpose:
        category = 'получение образования'
        return category
    else:
        category = 'операции с недвижимостью'
        return category

In [50]:
data['purpose_category'] = data['purpose'].apply(catagorize_purpose) 
data.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1.0,-8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1.0,-4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0.0,-5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3.0,-4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0.0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0.0,-926.185831,27,0,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0.0,-2879.202052,43,0,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0.0,-152.779569,50,1,0,M,сотрудник,0,135823,образование,C,получение образования
8,2.0,-6929.865299,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0.0,-2188.756445,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


In [51]:
data['have_kids'] = data['children']> 0

In [52]:
data_pivot = data.pivot_table(index=['children'], columns='gender', values='debt', aggfunc='sum')
data_pivot1 = data.pivot_table(index=['children'], columns='gender', values='debt', aggfunc='count')
data_pivot['all_sum'] = data_pivot.sum(axis=1)
data_pivot1['all_count'] = data_pivot1.sum(axis=1)
data_all = data_pivot.merge(data_pivot1, left_on='children', right_on='children', how='outer')
data_all['ratio'] = data_all['all_sum']/data_all['all_count']
data_all.head(10)
data_all.loc [:, 'ratio']

children
0.0    0.075272
1.0    0.092028
2.0    0.095553
3.0    0.082317
4.0    0.084507
5.0    0.000000
Name: ratio, dtype: float64

Судя по абсолютным данным чем больше детей тем меньше вероятность того, что ты просрочишь кредит. Причем эта предрасположенность распространяется и на мужчин и на женщин. Тем не менее при относительном рассмотрении цифры говорят другое - разницы примерно нет (порядка 10 процентов) исключение только люди у которых 5 детей - они самые ответсвенные.
Посмотрим в целом зависимость количества детей на просрочку кредита

In [53]:
data_pivot = data.pivot_table(index=['have_kids'], columns='gender', values='debt', aggfunc='sum')
data_pivot1 = data.pivot_table(index=['have_kids'], columns='gender', values='debt', aggfunc='count')
data_pivot['all_sum'] = data_pivot.sum(axis=1)
data_pivot1['all_count'] = data_pivot1.sum(axis=1)
data_all = data_pivot.merge(data_pivot1, left_on='have_kids', right_on='have_kids', how='outer')
data_all['ratio'] = data_all['all_sum']/data_all['all_count']
data_all.loc [:, 'ratio']

have_kids
False    0.075272
True     0.092349
Name: ratio, dtype: float64

Тут же можно сделать вывод, что андеррайтер скорее одобрит кредит человеку с детьми нежели без если он будет смотреть на абсолютыне числа и подумает что возможно наличие детей наделяет невобходимым уровнем ответственности для погашения кредита в срок. Но при просмотре относительных чисел он увидит, что вероятность отличается на 2 процента

In [54]:
data_final  = data.merge(family_status, on='family_status_id', how='left')

data_pivot = data_final.pivot_table(index=['family_status'], columns='gender', values='debt', aggfunc='sum')
data_pivot1 = data_final.pivot_table(index=['family_status'], columns='gender', values='debt', aggfunc='count')
data_pivot['all_sum'] = data_pivot.sum(axis=1)
data_pivot1['all_count'] = data_pivot1.sum(axis=1)
data_all = data_pivot.merge(data_pivot1, left_on='family_status', right_on='family_status', how='outer')
data_all['ratio'] = data_all['all_sum']/data_all['all_count']
data_all.loc [:, 'ratio']

family_status
в разводе                0.071130
вдовец / вдова           0.065693
гражданский брак         0.093471
женат / замужем          0.075452
не женат / не замужем    0.097509
Name: ratio, dtype: float64

Судя по абсолютным данным люди которые находятся без отношений наименне вероятно просрочат кредит. Относительные же дают повод порассуждать.
Так Люди состоящие в гражданском браке или не женатые/не зажужние вероятней просрочат кредит, а люди потерявщие супруга/супругу менее вероятней

In [55]:
data_pivot = data.pivot_table(index=['purpose_category'], columns='gender', values='debt', aggfunc='sum')
data_pivot = data_final.pivot_table(index=['purpose_category'], columns='gender', values='debt', aggfunc='sum')
data_pivot1 = data_final.pivot_table(index=['purpose_category'], columns='gender', values='debt', aggfunc='count')
data_pivot['all_sum'] = data_pivot.sum(axis=1)
data_pivot1['all_count'] = data_pivot1.sum(axis=1)
data_all = data_pivot.merge(data_pivot1, left_on='purpose_category', right_on='purpose_category', how='outer')
data_all['ratio'] = data_all['all_sum']/data_all['all_count']
data_all.loc [:, 'ratio']

purpose_category
операции с автомобилем      0.093590
операции с недвижимостью    0.072334
получение образования       0.092200
проведение свадьбы          0.080034
Name: ratio, dtype: float64

Подоавляющее количество "просрочек" по кредитам приходится на операции с недвижимостью, но это только потому что их их попросту больше, в относительном отношении выдача кредита для этой цели наиболее выгодна банку.

In [56]:
data_pivot = data.pivot_table(index=['total_income_category'], columns='gender', values='debt', aggfunc='sum')
data_pivot = data_final.pivot_table(index=['total_income_category'], columns='gender', values='debt', aggfunc='sum')
data_pivot1 = data_final.pivot_table(index=['total_income_category'], columns='gender', values='debt', aggfunc='count')
data_pivot['all_sum'] = data_pivot.sum(axis=1)
data_pivot1['all_count'] = data_pivot1.sum(axis=1)
data_all = data_pivot.merge(data_pivot1, left_on='total_income_category', right_on='total_income_category', how='outer')
data_all['ratio'] = data_all['all_sum']/data_all['all_count']
data_all.loc [:, 'ratio']

total_income_category
A    0.080000
B    0.070621
C    0.084915
D    0.060000
E    0.090909
Name: ratio, dtype: float64

Интересное заключение - люди с малым достатком чаще остальных просрачивают кредит, а люди зарабатывающие чуть выше малого меньше всего.

## Общий вывод:

Основной задачей данного иследования было вявление закономерности между разными факторами и возвратом кредита в срок.
Для проверки данной гипотезы использовалась выборка data.csv. Анализ данной выборки показал, что в файле присуствуют ошибки, которые были исправлены. Самая основная претензция к выборке - это отсутствие уникального идентификатора по записи, по которому и можно было бы определять дубликат. Аналогично тому как у family_status есть family_status_id.

Убрав все ошибки, обощая все собранные данные можно сказать, что важно смотреть на относительные данные ибо в противном случае банк будем думать что богатые или бедные люди с детьми и без отношений берущие кредит не для покупки недвижимости - самые лучшие клиенты банков, когда реальность немного сложнее. В прочем, ничего нового.